# Features extraction

In this file, the pre-trained model (https://github.com/csho33/bacteria-ID) is used as a features extractor.

## Librairies

In [1]:
from data_loader import load_data
import os, sys
import random
import numpy as np
import torch
from collections import OrderedDict
from extractor import Extractor
from time import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from statistics import mean

## Pre-trained models
In this part, the three provided pre-trained models are load.

In [2]:
os.chdir(os.getcwd())
models_dir = 'Bacteria_TL'
sys.path.append(models_dir)

Now, the import can be completed, using the file provided in the Bacterial_TL directory

In [3]:
from training import get_predictions
from datasets import spectral_dataloader

In [4]:
models = ['pretrained_model.ckpt', 'finetuned_model.ckpt', 'clinical_pretrained_model.ckpt']

## Dataset
This part is dedicated to the dataset. The data will be load and split into train, validation and test sets.

### Load data

In [5]:
X, Y, groups, _ = load_data()

### Split into train, validation and test sets
The split operation is doing as follow :
   - 60% of the patients data are take for the train set
   - 20% are take for the validation set
   - 20% are take for the test set

In [6]:
selected_groups = random.sample(range(0, 30), 30)
train_groups = selected_groups[:18]
validation_groups = selected_groups[18:24]
test_groups = selected_groups[24:]

In [7]:
def createSplit(X, Y, groups, groups_selected):
    X_split = []
    Y_split = []
    for i in range(len(X)):
        if(groups[i] in groups_selected):
            X_split.append(X[i])
            Y_split.append(Y[i])
    return np.array(X_split), np.array(Y_split)

In [8]:
X_train, Y_train = createSplit(X, Y, groups, train_groups)
X_validation, Y_validation = createSplit(X, Y, groups, validation_groups)
X_test, Y_test = createSplit(X, Y, groups, test_groups)

## Features extractors
This is the part where the different models are modified in order to be used as features extractor.

### Pre-trained models parameters
General parameters of pre-trained models are defined here.

In [9]:
base_layers = 6
hidden_size = 100
block_size = 2
input_dim = 1174
in_channels = 64
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()
base_to_remove = ['linear.weight', 'linear.bias']

### Utility functions

In [10]:
def removekey(d, block):
    r = dict(d)
    copy = r.copy()
    toRemove = [base_to_remove]
    if block != 0:
        toRemove.append('encoder.5')
    if block == 2:
        toRemove.append('encoder.4')
    
    for key in copy.keys():
        if (key in toRemove[0]):
            r.pop(key)
        if len(toRemove) > 1:
            if key.startswith(toRemove[1]):
                r.pop(key)
        if len(toRemove) == 3:
            if key.startswith(toRemove[2]):
                r.pop(key)
        
    return OrderedDict(r)

In [11]:
def load_model(modelToLoad, numberOfBlockToRemove):
    layers = base_layers - numberOfBlockToRemove
    hidden_sizes = [hidden_size] * layers
    num_blocks = [block_size] * layers
    model = Extractor(hidden_sizes, num_blocks)
    
    if cuda: model.cuda()
    
    checkpoint = torch.load(models_dir + '/' + modelToLoad, map_location=lambda storage, loc: storage)
    mod_weights = removekey(checkpoint, numberOfBlockToRemove)
    model.load_state_dict(mod_weights, strict=True)
    
    return model, mod_weights, checkpoint

### First experiment
Each pre-trained model is transformed in features extractor removing the final linear layer. 

#### "Pretrained model"

In [12]:
pretrainedNoLinear, _, _ = load_model(models[0], 0)

In [13]:
print(pretrainedNoLinear)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Finetuned model"

In [14]:
finetunedNoLinear, _, _ = load_model(models[1], 0)

In [15]:
print(finetunedNoLinear)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Clinical model"

In [16]:
clinicalNoLinear, _, _ = load_model(models[2], 0)

In [17]:
print(clinicalNoLinear)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

### Second experiment
Each pre-trained model is now transformed removing the last block of the original model.

#### "Pretrained model"

In [18]:
pretrainedNoLastBlock, _, _ = load_model(models[0], 1)

In [19]:
print(pretrainedNoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Finetuned model"

In [20]:
finetunedNoLastBlock, _, _ = load_model(models[1], 1)

In [21]:
print(finetunedNoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Clinical model"

In [22]:
clinicalNoLastBlock, _, _ = load_model(models[2], 1)

In [23]:
print(clinicalNoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

### Third experiment
Each pre-trained model is now transformed removing the last two blocks.

#### "Pretrained model"

In [24]:
pretrainedNoTwoLastBlock, _, _ = load_model(models[0], 2)

In [25]:
print(pretrainedNoTwoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Finetuned model"

In [26]:
finetunedNoTwoLastBlock, _, _ = load_model(models[1], 2)

In [27]:
print(finetunedNoTwoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Clinical model"

In [28]:
clinicalNoTwoLastBlock, _, _ = load_model(models[2], 2)

In [29]:
print(clinicalNoTwoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

## Features extraction
The features extractor created in the previous part are now going to be used.

In [30]:
dl_train = spectral_dataloader(X_train, Y_train, batch_size=10, shuffle=False)
dl_validation = spectral_dataloader(X_validation, Y_validation, batch_size=10, shuffle=False)
dl_test = spectral_dataloader(X_test, Y_test, batch_size=10, shuffle=False)

### Utility function

In [31]:
def groupFeatures(X, features):
    size = len(X)
    numFeatures = len(features)
    groupSize = numFeatures//size
    X_grouped = []
    num = 1
    tmp = []
    for i in range(numFeatures):
        if num > groupSize:
            num = 1
        if num == 1:
            tmp = []
        tmp.append(features[i])
        if num == groupSize:
            X_grouped.append(tmp)
        num+=1

    return X_grouped

### First category of extractors

In [32]:
#Train set & pretrainedNoLinear
t0 = time()
featuresPretrained1_train = get_predictions(pretrainedNoLinear, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained1_train)//len(X_train), time()-t0))
featuresPretrained1_train = groupFeatures(X_train, featuresPretrained1_train)
featuresPretrained1_train = np.array(featuresPretrained1_train)

#Validation set & pretrainedNoLinear
t0 = time()
featuresPretrained1_validation = get_predictions(pretrainedNoLinear, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained1_validation)//len(X_validation), time()-t0))
featuresPretrained1_validation = groupFeatures(X_validation, featuresPretrained1_validation)
featuresPretrained1_validation = np.array(featuresPretrained1_validation)

#Test set & pretrainedNoLinear
t0 = time()
featuresPretrained1_test = get_predictions(pretrainedNoLinear, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained1_test)//len(X_test), time()-t0))
featuresPretrained1_test = groupFeatures(X_test, featuresPretrained1_test)
featuresPretrained1_test = np.array(featuresPretrained1_test)

Predicted 37 features for each spectrum: 12.94s
Predicted 37 features for each spectrum: 5.40s
Predicted 37 features for each spectrum: 6.82s


In [33]:
#Train set & finetunedNoLinear
t0 = time()
featuresFinetuned1_train = get_predictions(finetunedNoLinear, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned1_train)//len(X_train), time()-t0))
featuresFinetuned1_train = groupFeatures(X_train, featuresFinetuned1_train)
featuresFinetuned1_train = np.array(featuresFinetuned1_train)

#Validation set & finetunedNoLinear
t0 = time()
featuresFinetuned1_validation = get_predictions(finetunedNoLinear, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned1_validation)//len(X_validation), time()-t0))
featuresFinetuned1_validation = groupFeatures(X_validation, featuresFinetuned1_validation)
featuresFinetuned1_validation = np.array(featuresFinetuned1_validation)

#Test set & finetunedNoLinear
t0 = time()
featuresFinetuned1_test = get_predictions(finetunedNoLinear, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned1_test)//len(X_test), time()-t0))
featuresFinetuned1_test = groupFeatures(X_test, featuresFinetuned1_test)
featuresFinetuned1_test = np.array(featuresFinetuned1_test)

Predicted 37 features for each spectrum: 13.64s
Predicted 37 features for each spectrum: 7.60s
Predicted 37 features for each spectrum: 7.18s


In [34]:
#Train set & clinicalNoLinear
t0 = time()
featuresClinical1_train = get_predictions(clinicalNoLinear, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical1_train)//len(X_train), time()-t0))
featuresClinical1_train = groupFeatures(X_train, featuresClinical1_train)
featuresClinical1_train = np.array(featuresClinical1_train)

#Validation set & clinicalNoLinear
t0 = time()
featuresClinical1_validation = get_predictions(clinicalNoLinear, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical1_validation)//len(X_validation), time()-t0))
featuresClinical1_validation = groupFeatures(X_validation, featuresClinical1_validation)
featuresClinical1_validation = np.array(featuresClinical1_validation)

#Test set & clinicalNoLinear
t0 = time()
featuresClinical1_test = get_predictions(clinicalNoLinear, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical1_test)//len(X_test), time()-t0))
featuresClinical1_test = groupFeatures(X_test, featuresClinical1_test)
featuresClinical1_test = np.array(featuresClinical1_test)

Predicted 37 features for each spectrum: 11.90s
Predicted 37 features for each spectrum: 5.04s
Predicted 37 features for each spectrum: 7.27s


### Second category of extractors

In [35]:
#Train set & pretrainedNoLastBlock
t0 = time()
featuresPretrained2_train = get_predictions(pretrainedNoLastBlock, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained2_train)//len(X_train), time()-t0))
featuresPretrained2_train = groupFeatures(X_train, featuresPretrained2_train)
featuresPretrained2_train = np.array(featuresPretrained2_train)

#Validation set & pretrainedNoLastBlock
t0 = time()
featuresPretrained2_validation = get_predictions(pretrainedNoLastBlock, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained2_validation)//len(X_validation), time()-t0))
featuresPretrained2_validation = groupFeatures(X_validation, featuresPretrained2_validation)
featuresPretrained2_validation = np.array(featuresPretrained2_validation)

#Test set & pretrainedNoLastBlock
t0 = time()
featuresPretrained2_test = get_predictions(pretrainedNoLastBlock, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained2_test)//len(X_test), time()-t0))
featuresPretrained2_test = groupFeatures(X_test, featuresPretrained2_test)
featuresPretrained2_test = np.array(featuresPretrained2_test)

Predicted 74 features for each spectrum: 11.46s
Predicted 74 features for each spectrum: 4.89s
Predicted 74 features for each spectrum: 8.53s


In [36]:
#Train set & finetunedNoLastBlock
t0 = time()
featuresFinetuned2_train = get_predictions(finetunedNoLastBlock, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned2_train)//len(X_train), time()-t0))
featuresFinetuned2_train = groupFeatures(X_train, featuresFinetuned2_train)
featuresFinetuned2_train = np.array(featuresFinetuned2_train)

#Validation set & finetunedNoLastBlock
t0 = time()
featuresFinetuned2_validation = get_predictions(finetunedNoLastBlock, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned2_validation)//len(X_validation), time()-t0))
featuresFinetuned2_validation = groupFeatures(X_validation, featuresFinetuned2_validation)
featuresFinetuned2_validation = np.array(featuresFinetuned2_validation)

#Test set & finetunedNoLastBlock
t0 = time()
featuresFinetuned2_test = get_predictions(finetunedNoLastBlock, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned2_test)//len(X_test), time()-t0))
featuresFinetuned2_test = groupFeatures(X_test, featuresFinetuned2_test)
featuresFinetuned2_test = np.array(featuresFinetuned2_test)

Predicted 74 features for each spectrum: 13.65s
Predicted 74 features for each spectrum: 5.35s
Predicted 74 features for each spectrum: 7.35s


In [37]:
#Train set & clinicalNoLastBlock
t0 = time()
featuresClinical2_train = get_predictions(clinicalNoLastBlock, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical2_train)//len(X_train), time()-t0))
featuresClinical2_train = groupFeatures(X_train, featuresClinical2_train)
featuresClinical2_train = np.array(featuresClinical2_train)

#Validation set & clinicalNoLastBlock
t0 = time()
featuresClinical2_validation = get_predictions(clinicalNoLastBlock, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical2_validation)//len(X_validation), time()-t0))
featuresClinical2_validation = groupFeatures(X_validation, featuresClinical2_validation)
featuresClinical2_validation = np.array(featuresClinical2_validation)

#Test set & clinicalNoLastBlock
t0 = time()
featuresClinical2_test = get_predictions(clinicalNoLastBlock, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical2_test)//len(X_test), time()-t0))
featuresClinical2_test = groupFeatures(X_test, featuresClinical2_test)
featuresClinical2_test = np.array(featuresClinical2_test)

Predicted 74 features for each spectrum: 13.54s
Predicted 74 features for each spectrum: 5.26s
Predicted 74 features for each spectrum: 7.54s


### Third category of extractors

In [38]:
#Train set & pretrainedNoTwoLastBlock
t0 = time()
featuresPretrained3_train = get_predictions(pretrainedNoTwoLastBlock, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained3_train)//len(X_train), time()-t0))
featuresPretrained3_train = groupFeatures(X_train, featuresPretrained3_train)
featuresPretrained3_train = np.array(featuresPretrained3_train)

#Validation set & pretrainedNoTwoLastBlock
t0 = time()
featuresPretrained3_validation = get_predictions(pretrainedNoTwoLastBlock, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained3_validation)//len(X_validation), time()-t0))
featuresPretrained3_validation = groupFeatures(X_validation, featuresPretrained3_validation)
featuresPretrained3_validation = np.array(featuresPretrained3_validation)

#Test set & pretrainedNoTwoLastBlock
t0 = time()
featuresPretrained3_test = get_predictions(pretrainedNoTwoLastBlock, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresPretrained3_test)//len(X_test), time()-t0))
featuresPretrained3_test = groupFeatures(X_test, featuresPretrained3_test)
featuresPretrained3_test = np.array(featuresPretrained3_test)

Predicted 147 features for each spectrum: 17.80s
Predicted 147 features for each spectrum: 5.23s
Predicted 147 features for each spectrum: 7.09s


In [39]:
#Train set & finetunedNoTwoLastBlock
t0 = time()
featuresFinetuned3_train = get_predictions(finetunedNoTwoLastBlock, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned3_train)//len(X_train), time()-t0))
featuresFinetuned3_train = groupFeatures(X_train, featuresFinetuned3_train)
featuresFinetuned3_train = np.array(featuresFinetuned3_train)

#Validation set & finetunedNoTwoLastBlock
t0 = time()
featuresFinetuned3_validation = get_predictions(finetunedNoTwoLastBlock, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned3_validation)//len(X_validation), time()-t0))
featuresFinetuned3_validation = groupFeatures(X_validation, featuresFinetuned3_validation)
featuresFinetuned3_validation = np.array(featuresFinetuned3_validation)

#Test set & finetunedNoTwoLastBlock
t0 = time()
featuresFinetuned3_test = get_predictions(finetunedNoTwoLastBlock, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresFinetuned3_test)//len(X_test), time()-t0))
featuresFinetuned3_test = groupFeatures(X_test, featuresFinetuned3_test)
featuresFinetuned3_test = np.array(featuresFinetuned3_test)

Predicted 147 features for each spectrum: 12.92s
Predicted 147 features for each spectrum: 5.22s
Predicted 147 features for each spectrum: 7.14s


In [40]:
#Train set & clinicalNoTwoLastBlock
t0 = time()
featuresClinical3_train = get_predictions(clinicalNoTwoLastBlock, dl_train, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical3_train)//len(X_train), time()-t0))
featuresClinical3_train = groupFeatures(X_train, featuresClinical3_train)
featuresClinical3_train = np.array(featuresClinical3_train)

#Validation set & clinicalNoTwoLastBlock
t0 = time()
featuresClinical3_validation = get_predictions(clinicalNoTwoLastBlock, dl_validation, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical3_validation)//len(X_validation), time()-t0))
featuresClinical3_validation = groupFeatures(X_validation, featuresClinical3_validation)
featuresClinical3_validation = np.array(featuresClinical3_validation)

#Test set & clinicalNoTwoLastBlock
t0 = time()
featuresClinical3_test = get_predictions(clinicalNoTwoLastBlock, dl_test, cuda)
print('Predicted {} features for each spectrum: {:0.2f}s'.format(len(featuresClinical3_test)//len(X_test), time()-t0))
featuresClinical3_test = groupFeatures(X_test, featuresClinical3_test)
featuresClinical3_test = np.array(featuresClinical3_test)

Predicted 147 features for each spectrum: 14.01s
Predicted 147 features for each spectrum: 6.35s
Predicted 147 features for each spectrum: 7.24s


## Predict using "classical" model

### Decision Tree Classifier

#### Utility function

In [41]:
def predictWithDTC(X_train, Y_train, X_validation, Y_validation, X_test, Y_test):
    model = DecisionTreeClassifier()
    model = model.fit(X_train, Y_train)
    val_score = cross_val_score(model, X_validation, Y_validation, scoring='accuracy')
    prediction = model.predict(X_test)
    accuracy_test = accuracy_score(Y_test, prediction)
    return val_score, accuracy_test

#### With features from first category of extractor

In [42]:
val_score_pretrained1, accuracy_score_pretrained1 = predictWithDTC(featuresPretrained1_train, Y_train, featuresPretrained1_validation, Y_validation, featuresPretrained1_test, Y_test)
print('Using simple pretrained model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_pretrained1), accuracy_score_pretrained1))

Using simple pretrained model as features etractor:
	Accuracy obtained on validation set: 0.90000
	Accuracy obtained on test set: 0.54839


In [43]:
val_score_finetuned1, accuracy_score_finetuned1 = predictWithDTC(featuresFinetuned1_train, Y_train, featuresFinetuned1_validation, Y_validation, featuresFinetuned1_test, Y_test)
print('Using fituned model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_finetuned1), accuracy_score_finetuned1))

Using fituned model as features etractor:
	Accuracy obtained on validation set: 0.84667
	Accuracy obtained on test set: 0.67097


In [44]:
val_score_clinical1, accuracy_score_clinical1 = predictWithDTC(featuresClinical1_train, Y_train, featuresClinical1_validation, Y_validation, featuresClinical1_test, Y_test)
print('Using clinical model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_clinical1), accuracy_score_clinical1))

Using clinical model as features etractor:
	Accuracy obtained on validation set: 0.90190
	Accuracy obtained on test set: 0.60000


#### With features from second category of extractors

In [45]:
val_score_pretrained2, accuracy_score_pretrained2 = predictWithDTC(featuresPretrained2_train, Y_train, featuresPretrained2_validation, Y_validation, featuresPretrained2_test, Y_test)
print('Using simple pretrained model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_pretrained2), accuracy_score_pretrained2))

Using simple pretrained model as features etractor:
	Accuracy obtained on validation set: 0.90286
	Accuracy obtained on test set: 0.78710


In [46]:
val_score_finetuned2, accuracy_score_finetuned2 = predictWithDTC(featuresFinetuned2_train, Y_train, featuresFinetuned2_validation, Y_validation, featuresFinetuned2_test, Y_test)
print('Using fituned model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_finetuned2), accuracy_score_finetuned2))

Using fituned model as features etractor:
	Accuracy obtained on validation set: 0.95810
	Accuracy obtained on test set: 0.74194


In [47]:
val_score_clinical2, accuracy_score_clinical2 = predictWithDTC(featuresClinical2_train, Y_train, featuresClinical2_validation, Y_validation, featuresClinical2_test, Y_test)
print('Using clinical model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_clinical2), accuracy_score_clinical2))

Using clinical model as features etractor:
	Accuracy obtained on validation set: 0.78667
	Accuracy obtained on test set: 0.63226


#### With features from third category of extractors

In [48]:
val_score_pretrained3, accuracy_score_pretrained3 = predictWithDTC(featuresPretrained3_train, Y_train, featuresPretrained3_validation, Y_validation, featuresPretrained3_test, Y_test)
print('Using simple pretrained model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_pretrained3), accuracy_score_pretrained3))

Using simple pretrained model as features etractor:
	Accuracy obtained on validation set: 0.94381
	Accuracy obtained on test set: 0.84516


In [49]:
val_score_finetuned3, accuracy_score_finetuned3 = predictWithDTC(featuresFinetuned3_train, Y_train, featuresFinetuned3_validation, Y_validation, featuresFinetuned3_test, Y_test)
print('Using fituned model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_finetuned3), accuracy_score_finetuned3))

Using fituned model as features etractor:
	Accuracy obtained on validation set: 0.95714
	Accuracy obtained on test set: 0.69032


In [50]:
val_score_clinical3, accuracy_score_clinical3 = predictWithDTC(featuresClinical3_train, Y_train, featuresClinical3_validation, Y_validation, featuresClinical3_test, Y_test)
print('Using clinical model as features etractor:\n\tAccuracy obtained on validation set: {:0.5f}\n\tAccuracy obtained on test set: {:0.5f}'.format(mean(val_score_clinical3), accuracy_score_clinical3))

Using clinical model as features etractor:
	Accuracy obtained on validation set: 0.94286
	Accuracy obtained on test set: 0.65806
